# Building hotel room search with self-querying retrieval

In this example we'll walk through how to build and iterate on a hotel room search service that leverages an LLM to generate structured filter queries that can then be passed to a vector store.

For an introduction to self-querying retrieval [check out the docs](https://python.langchain.com/docs/modules/data_connection/retrievers/self_query).

## Imports and data prep

In this example we use `ChatOpenAI` for the model and `ElasticsearchStore` for the vector store, but these can be swapped out with an LLM/ChatModel and [any VectorStore that support self-querying](https://python.langchain.com/docs/integrations/retrievers/self_query/).

Download data from: https://www.kaggle.com/datasets/keshavramaiah/hotel-recommendation

In [ ]:
!pip install langchain lark openai elasticsearch pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.2/220.2 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 re

In [1]:
import pandas as pd

In [3]:
details = (
    pd.read_csv("/content/Hotel_details.csv")
    .drop_duplicates(subset="hotelid")
    .set_index("hotelid")
)
attributes = pd.read_csv(
    "./Hotel_Room_attributes.csv", index_col="id"
)
price = pd.read_csv("./hotels_RoomPrice.csv", index_col="id")

In [4]:
latest_price = price.drop_duplicates(subset="refid", keep="last")[
    [
        "hotelcode",
        "roomtype",
        "onsiterate",
        "roomamenities",
        "maxoccupancy",
        "mealinclusiontype",
    ]
]
latest_price["ratedescription"] = attributes.loc[latest_price.index]["ratedescription"]
latest_price = latest_price.join(
    details[["hotelname", "city", "country", "starrating"]], on="hotelcode"
)
latest_price = latest_price.rename({"ratedescription": "roomdescription"}, axis=1)
latest_price["mealsincluded"] = ~latest_price["mealinclusiontype"].isnull()
latest_price.pop("hotelcode")
latest_price.pop("mealinclusiontype")
latest_price = latest_price.reset_index(drop=True)
latest_price.head()

,roomtype,onsiterate,roomamenities,maxoccupancy,roomdescription,hotelname,city,country,starrating,mealsincluded
0,Vacation Home,636.09,Air conditioning: ;Closet: ;Fireplace: ;Free W...,4.0,"Shower, Kitchenette, 2 bedrooms, 1 double bed ...",Pantlleni,Beddgelert,United Kingdom,3.0,False
1,Vacation Home,591.74,Air conditioning: ;Closet: ;Dishwasher: ;Firep...,4.0,"Shower, Kitchenette, 2 bedrooms, 1 double bed ...",Willow Cottage,Beverley,United Kingdom,3.0,False
2,"Guest room, Queen or Twin/Single Bed(s)",0.00,NaN,2.0,NaN,AC Hotel Manchester Salford Quays,Manchester,United Kingdom,4.0,False
3,Bargemaster King Accessible Room,379.08,Air conditioning: ;Free Wi-Fi in all rooms!: ;...,2.0,Shower,NaN,NaN,NaN,NaN,True
4,Twin Room,156.17,Additional toilet: ;Air conditioning: ;Blackou...,2.0,"Room size: 15 m²/161 ft², Non-smoking, Shower,...",Ibis London Canning Town,London,United Kingdom,3.0,True


## Describe data attributes

We'll use a self-query retriever, which requires us to describe the metadata we can filter on.

Or if we're feeling lazy we can have a model write a draft of the descriptions for us :)

In [6]:
from google.colab import userdata


api_key = userdata.get('us_api_key')
base_url = userdata.get('us_url')

In [15]:
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=api_key, base_url=base_url, model="gpt-4")
res = model.predict(
    "Below is a table with information about hotel rooms. "
    "Return a JSON list with an entry for each column. Each entry should have "
    '{"name": "column name", "description": "column description", "type": "column data type"}'
    f"\n\n{latest_price.head()}\n\nJSON:\n"
)

In [18]:
import json

attribute_info = json.loads(res)
attribute_info

[{'name': 'roomtype',
  'description': 'The type of room available in the hotel',
  'type': 'string'},
 {'name': 'onsiterate',
  'description': 'The rate of the room per night in dollars',
  'type': 'float'},
 {'name': 'roomamenities',
  'description': 'Amenities available in the room',
  'type': 'string'},
 {'name': 'maxoccupancy',
  'description': 'The maximum number of people that can occupy the room',
  'type': 'float'},
 {'name': 'roomdescription',
  'description': 'A description of the room',
  'type': 'string'},
 {'name': 'hotelname',
  'description': 'The name of the hotel',
  'type': 'string'},
 {'name': 'city',
  'description': 'The city where the hotel is located',
  'type': 'string'},
 {'name': 'country',
  'description': 'The country where the hotel is located',
  'type': 'string'},
 {'name': 'starrating',
  'description': 'The star rating of the hotel',
  'type': 'float'},
 {'name': 'mealsincluded',
  'description': 'Whether meals are included in the room rate',
  'type':

For low cardinality features, let's include the valid values in the description

In [19]:
latest_price.nunique()[latest_price.nunique() < 40]

maxoccupancy      9
country          24
starrating        3
mealsincluded     2
dtype: int64

In [20]:
attribute_info[-2][
    "description"
] += f". Valid values are {sorted(latest_price['starrating'].value_counts().index.tolist())}"
attribute_info[3][
    "description"
] += f". Valid values are {sorted(latest_price['maxoccupancy'].value_counts().index.tolist())}"
attribute_info[-3][
    "description"
] += f". Valid values are {sorted(latest_price['country'].value_counts().index.tolist())}"

In [21]:
attribute_info

[{'name': 'roomtype',
  'description': 'The type of room available in the hotel',
  'type': 'string'},
 {'name': 'onsiterate',
  'description': 'The rate of the room per night in dollars',
  'type': 'float'},
 {'name': 'roomamenities',
  'description': 'Amenities available in the room',
  'type': 'string'},
 {'name': 'maxoccupancy',
  'description': 'The maximum number of people that can occupy the room. Valid values are [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 10.0]',
  'type': 'float'},
 {'name': 'roomdescription',
  'description': 'A description of the room',
  'type': 'string'},
 {'name': 'hotelname',
  'description': 'The name of the hotel',
  'type': 'string'},
 {'name': 'city',
  'description': 'The city where the hotel is located',
  'type': 'string'},
 {'name': 'country',
  'description': "The country where the hotel is located. Valid values are ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic', 'Finland', 'France', 'Germany', 'Greece', 'Italy', 'Latvia'

## Creating a query constructor chain

Let's take a look at the chain that will convert natural language requests into structured queries.

To start we can just load the prompt and see what it looks like

In [22]:
from langchain.chains.query_constructor.base import (
    get_query_constructor_prompt,
    load_query_constructor_runnable,
)

In [23]:
doc_contents = "Detailed description of a hotel room"
prompt = get_query_constructor_prompt(doc_contents, attribute_info)
print(prompt.format(query="{query}"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

In [24]:
chain = load_query_constructor_runnable(
    ChatOpenAI(openai_api_key=api_key, base_url=base_url , model="gpt-4-32k", temperature=0), doc_contents, attribute_info
)

In [25]:
chain.invoke({"query": "I want a hotel in Southern Europe and my budget is 200 bucks."})

StructuredQuery(query='hotel in Southern Europe', filter=Comparison(comparator=<Comparator.LTE: 'lte'>, attribute='onsiterate', value=200), limit=None)

In [26]:
chain.invoke(
    {
        "query": "Find a 2-person room in Vienna or London, preferably with meals included and AC"
    }
)

StructuredQuery(query='AC', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='maxoccupancy', value=2.0), Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='city', value='Vienna'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='city', value='London')]), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='mealsincluded', value=True)]), limit=None)

## Refining attribute descriptions

We can see that at least two issues above. First is that when we ask for a Southern European destination we're only getting a filter for Italy, and second when we ask for AC we get a literal string lookup for AC (which isn't so bad but will miss things like 'Air conditioning').

As a first step, let's try to update our description of the 'country' attribute to emphasize that equality should only be used when a specific country is mentioned.

In [28]:
attribute_info[-3][
    "description"
] += ". NOTE: Only use the 'eq' operator if a specific country is mentioned. If a region is mentioned, include all relevant countries in filter."
chain = load_query_constructor_runnable(
    ChatOpenAI(openai_api_key=api_key, base_url=base_url , model="gpt-4-32k", temperature=0),
    doc_contents,
    attribute_info,
)

In [29]:
chain.invoke({"query": "I want a hotel in Southern Europe and my budget is 200 bucks."})

StructuredQuery(query='hotel', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.IN: 'in'>, attribute='country', value=['Italy', 'Greece', 'Spain', 'Portugal', 'Malta', 'Cyprus', 'Slovenia', 'Croatia']), Comparison(comparator=<Comparator.LTE: 'lte'>, attribute='onsiterate', value=200)]), limit=None)

## Refining which attributes to filter on

This seems to have helped! Now let's try to narrow the attributes we're filtering on. More freeform attributes we can leave to the main query, which is better for capturing semantic meaning than searching for specific substrings.

In [32]:
content_attr = ["roomtype", "roomamenities", "roomdescription", "hotelname"]
doc_contents = "A detailed description of a hotel room, including information about the room type and room amenities."
filter_attribute_info = tuple(
    ai for ai in attribute_info if ai["name"] not in content_attr
)
chain = load_query_constructor_runnable(
    ChatOpenAI(openai_api_key=api_key, base_url=base_url , model="gpt-4-32k", temperature=0),
    doc_contents,
    filter_attribute_info,
)

In [33]:
chain.invoke(
    {
        "query": "Find a 2-person room in Vienna or London, preferably with meals included and AC"
    }
)

StructuredQuery(query='2-person room with meals included and AC', filter=Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='city', value='Vienna'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='city', value='London')]), limit=None)

## Adding examples specific to our use case

We've removed the strict filter for 'AC' but it's still not being included in the query string. Our chain prompt is a few-shot prompt with some default examples. Let's see if adding use case-specific examples will help:

In [34]:
examples = [
    (
        "I want a hotel in the Balkans with a king sized bed and a hot tub. Budget is $300 a night",
        {
            "query": "king-sized bed, hot tub",
            "filter": 'and(in("country", ["Bulgaria", "Greece", "Croatia", "Serbia"]), lte("onsiterate", 300))',
        },
    ),
    (
        "A room with breakfast included for 3 people, at a Hilton",
        {
            "query": "Hilton",
            "filter": 'and(eq("mealsincluded", true), gte("maxoccupancy", 3))',
        },
    ),
]
prompt = get_query_constructor_prompt(
    doc_contents, filter_attribute_info, examples=examples
)
print(prompt.format(query="{query}"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

In [38]:
chain = load_query_constructor_runnable(
    ChatOpenAI(openai_api_key=api_key, base_url=base_url , model="gpt-4-32k", temperature=0),
    doc_contents,
    filter_attribute_info,
    examples=examples,
)

In [39]:
chain.invoke(
    {
        "query": "Find a 2-person room in Vienna or London, preferably with meals included and AC"
    }
)

StructuredQuery(query='2-person room, AC', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Operation(operator=<Operator.OR: 'or'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='city', value='Vienna'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='city', value='London')]), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='mealsincluded', value=True)]), limit=None)

This seems to have helped! Let's try another complex query:

In [40]:
chain.invoke(
    {
        "query": "I want to stay somewhere highly rated along the coast. I want a room with a patio and a fireplace."
    }
)

StructuredQuery(query='coast, patio, fireplace', filter=Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='starrating', value=4), limit=None)

## Automatically ignoring invalid queries

It seems our model get's tripped up on this more complex query and tries to search over an attribute ('description') that doesn't exist. By setting `fix_invalid=True` in our query constructor chain, we can automatically remove any parts of the filter that is invalid (meaning it's using disallowed operations, comparisons or attributes).

In [41]:
chain = load_query_constructor_runnable(
    ChatOpenAI(openai_api_key=api_key, base_url=base_url , model="gpt-4-32k", temperature=0),
    doc_contents,
    filter_attribute_info,
    examples=examples,
    fix_invalid=True,
)

In [42]:
chain.invoke(
    {
        "query": "I want to stay somewhere highly rated along the coast. I want a room with a patio and a fireplace."
    }
)

StructuredQuery(query='coast, patio, fireplace', filter=Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='starrating', value=4.0), limit=None)

## Using with a self-querying retriever

Now that our query construction chain is in a decent place, let's try using it with an actual retriever. For this example we'll use the [ElasticsearchStore](https://python.langchain.com/docs/integrations/vectorstores/elasticsearch).

In [43]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.vectorstores import ElasticsearchStore

llm = ChatOpenAI(openai_api_key=api_key, base_url=base_url , model="gpt-4-32k", temperature=0)
embeddings = OpenAIEmbeddings(openai_api_key=userdata.get('OPENAI_API_KEY'))

## Populating vectorstore

The first time you run this, uncomment the below cell to first index the data.

In [ ]:
# docs = []
# for _, room in latest_price.fillna("").iterrows():
#     doc = Document(
#         page_content=json.dumps(room.to_dict(), indent=2),
#         metadata=room.to_dict()
#     )
#     docs.append(doc)
# vecstore = ElasticsearchStore.from_documents(
#     docs,
#     embeddings,
#     es_url="http://localhost:9200",
#     index_name="hotel_rooms",
#     # strategy=ElasticsearchStore.ApproxRetrievalStrategy(
#     #     hybrid=True,
#     # )
# )

In [ ]:
vecstore = ElasticsearchStore(
    "hotel_rooms",
    embedding=embeddings,
    es_url="http://localhost:9200",
    # strategy=ElasticsearchStore.ApproxRetrievalStrategy(hybrid=True) # seems to not be available in community version
)

In [ ]:
from langchain.retrievers import SelfQueryRetriever

retriever = SelfQueryRetriever(
    query_constructor=chain, vectorstore=vecstore, verbose=True
)

In [ ]:
results = retriever.get_relevant_documents(
    "I want to stay somewhere highly rated along the coast. I want a room with a patio and a fireplace."
)
for res in results:
    print(res.page_content)
    print("\n" + "-" * 20 + "\n")

{
  "roomtype": "Three-Bedroom House With Sea View",
  "onsiterate": 341.75,
  "roomamenities": "Additional bathroom: ;Additional toilet: ;Air conditioning: ;Closet: ;Clothes dryer: ;Coffee/tea maker: ;Dishwasher: ;DVD/CD player: ;Fireplace: ;Free Wi-Fi in all rooms!: ;Full kitchen: ;Hair dryer: ;Heating: ;High chair: ;In-room safe box: ;Ironing facilities: ;Kitchenware: ;Linens: ;Microwave: ;Private entrance: ;Refrigerator: ;Seating area: ;Separate dining area: ;Smoke detector: ;Sofa: ;Towels: ;TV [flat screen]: ;Washing machine: ;",
  "maxoccupancy": 6,
  "roomdescription": "Room size: 125 m\u00b2/1345 ft\u00b2, 2 bathrooms, Shower and bathtub, Shared bathroom, Kitchenette, 3 bedrooms, 1 double bed or 2 single beds or 1 double bed",
  "hotelname": "Downings Coastguard Cottages - Type B-E",
  "city": "Downings",
  "country": "Ireland",
  "starrating": 4,
  "mealsincluded": false
}

--------------------

{
  "roomtype": "Three-Bedroom House With Sea View",
  "onsiterate": 774.05,
  "ro